### 환경 불러오기

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
# copywarning ignore
import warnings
#from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from multiprocessing import Process, Value, Array

### 오피스텔 실거래 데이터 불러오기(2022.03 ~ 2023.02)

In [2]:
# officetels 폴더에 있는 모든 파일명 리스트로 저장
off_forders = os.listdir('../data/officetels')
# csv 파일을 모두 불러와서 병합
df_officetels = pd.DataFrame()

#### 오피스텔 도로명 주소 Geocoding - 위도, 경도 추출

In [3]:
api_key = '79d3d40921cad65ee45ee1059c974972' # daero
# api_key = 'a6bd28c812cc2401914de6d7fcc93171' # jihwan

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['documents'][0]['road_address']
        return float(match_first['x']), float(match_first['y'])
    except:
        return 0, 0

def get_lat_lon(df):
    for i in range(0, len(df)):
        try:
            addr = df.iloc[i]['시군구'] + " " + df.iloc[i]['도로명']
            x, y = addr_to_lat_lon(addr)
        except:
            x, y = 0, 0
        df['longitude'].iloc[i] = x
        df['latitude'].iloc[i] = y
    

### 카카오 rest api로 lat, lon 받아오기

In [4]:
# for i in range(0, len(off_forders)):
#     if off_forders[i].split('.')[1] == 'csv':
#         file = '../data/officetels/'+off_forders[i]
#         # csv파일의 불필요한 주석정보 제거
#         sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
#         sub_df['도로명'].replace(' ',np.nan, inplace=True)
#         sub_df.dropna(subset=['도로명'],axis=0,inplace=True)
#         sub_df.insert(14,'latitude',[np.NaN for k in range(0,len(sub_df))])
#         sub_df.insert(14,'longitude',[np.NaN for k in range(0,len(sub_df))])
#         get_lat_lon(sub_df)
#         # concat함수로 행을 추가하며 병합
#         sub_df.to_csv(file, encoding='CP949',index=False)
#         df_officetels = pd.concat([df_officetels, sub_df], ignore_index=True)
#         print(i,'complete')

# df_officetels.to_csv(path_or_buf='오피스텔 실거래가_22-03_23-02', encoding='CP949')

In [13]:
df_officetels = pd.read_csv('../data/오피스텔 실거래가_22-03_23-02.csv', encoding='CP949')
# 위경도가 주소가 없는 행은 제거
df_officetels = df_officetels[df_officetels['longitude']!=0].reset_index(drop=True)
df_officetels

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_10360\725463772.py:1: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_officetels = pd.read_csv('../data/오피스텔 실거래가_22-03_23-02.csv', encoding='CP949')


,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),...,층,건축년도,도로명,latitude,longitude,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원)
0,강원도 강릉시 교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,11,400,...,7,2020.0,강릉대로 285,37.762775,128.899711,202203~202303,신규,-,0,0.0
1,강원도 강릉시 교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,19,500,...,13,2020.0,강릉대로 285,37.762775,128.899711,202206~202305,신규,-,0,0.0
2,강원도 강릉시 교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,23,50,...,11,2020.0,강릉대로 285,37.762775,128.899711,202203~202304,신규,-,0,0.0
3,강원도 강릉시 교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,25,500,...,11,2020.0,강릉대로 285,37.762775,128.899711,202204~202404,신규,-,0,0.0
4,강원도 강릉시 교동,140-16,140,16,강릉역 블루핀,월세,22.00,202203,27,500,...,11,2020.0,강릉대로 285,37.762775,128.899711,202205~202305,신규,-,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164182,충청북도 청주흥덕구 오송읍 연제리,664,664,0,두산위브 센티움,월세,28.90,202302,28,450,...,5,2014.0,오송생명3로 31,36.635157,127.328279,202303~202403,갱신,-,331,43.0
164183,충청북도 청주흥덕구 외북동,317,317,0,아크로폴리스,전세,20.40,202302,4,"5,000",...,7,2019.0,엘지로 23,36.660285,127.435104,-,-,-,NaN,NaN
164184,충청북도 청주흥덕구 외북동,317,317,0,아크로폴리스,전세,37.05,202302,4,"10,000",...,5,2019.0,엘지로 23,36.660285,127.435104,202302~202502,신규,-,0,0.0
164185,충청북도 충주시 주덕읍 화곡리,1207,1207,0,시그니처시티,월세,18.82,202302,7,500,...,10,2020.0,기업도시2로 152,37.004184,127.816580,202302~202402,신규,-,0,0.0


In [14]:
# df_officetels.to_json('../data/json/officetels.json')

In [6]:
class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"
    
        # 카테고리 검색 메서드
    def search_category(self, category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
            """
            06 카테고리 검색
            """
            params = {'category_group_code': f"{category_group_code}",
                    'x': f"{x}",
                    'y': f"{y}"}
            
            if radius != None:
                params['radius'] = f"{radius}"
            if rect != None:
                params['rect'] = f"{rect}"
            if page != None:
                params['page'] = f"{page}"
            if size != None:
                params['size'] = f"{size}"
            if sort != None:
                params['sort'] = f"{sort}"
                
            res = requests.get(self.URL_06, headers=self.headers, params=params)
            document = json.loads(res.text)
            
            return document

### 카테고리별 주변 정보 탐색

In [7]:
# REST API 키
rest_api_key = "79d3d40921cad65ee45ee1059c974972"

kakao = KakaoLocalAPI(rest_api_key)

In [8]:
unique_df_officetels = df_officetels.drop_duplicates('도로명', keep='first', ignore_index=True)
len(unique_df_officetels)

9750

In [9]:
# def get_category(addr, lat, lon):
#     try:
#         # 편의점 : CS2
#         cs2 = kakao.search_category('CS2', lat, lon, 300)
#         # 학교 : SC4
#         sc4 = kakao.search_category('SC4', lat, lon, 300)
#         # 지하철역 : SW8
#         sw8 = kakao.search_category('SW8', lat, lon, 300)
#         # 음식점 : FD6
#         fd6 = kakao.search_category('FD6', lat, lon, 300)
#         # 카페 : CE7
#         ce7 = kakao.search_category('CE7', lat, lon, 300)
#         # 병원 : HP8
#         hp8 = kakao.search_category('HP8', lat, lon, 300)
#         # 약국 : PM9
#         pm9 = kakao.search_category('PM9', lat, lon, 300)

#         category = {"CS2": [cs2["documents"]], "SC4": [sc4["documents"]], "SW8": [sw8["documents"]], "FD6": [fd6["documents"]], "CE7": [ce7["documents"]], "HP8": [hp8["documents"]], "PM9": [pm9["documents"]]}

#         around_data_json['officetels'].append({
#             "address": addr,
#             "latitude": lat,
#             "longitude": lon,
#             "category": [category]
#         })
#         print(category) 
#     except:     
#         print(addr)
        

# around_data_json = {}
# around_data_json['officetels'] = []

# addr = unique_df_officetels['도로명'].to_list()
# lat = unique_df_officetels['latitude'].to_list()
# lon = unique_df_officetels['longitude'].to_list()

# process = Process(target=get_category, args=(addr, lat, lon)) 
# process.start()
# process.join()

# around_data_json


In [10]:
around_data_json = {}
around_data_json['officetels'] = []

for i in range(0, len(unique_df_officetels)):
    addr = unique_df_officetels.iloc[i]['도로명']
    lat = unique_df_officetels.iloc[i]['latitude']
    lon = unique_df_officetels.iloc[i]['longitude']

    try:
        # 편의점 : CS2
        cs2 = kakao.search_category('CS2', lat, lon, 300)
        # 학교 : SC4
        sc4 = kakao.search_category('SC4', lat, lon, 300)
        # 지하철역 : SW8
        sw8 = kakao.search_category('SW8', lat, lon, 300)
        # 음식점 : FD6
        fd6 = kakao.search_category('FD6', lat, lon, 300)
        # 카페 : CE7
        ce7 = kakao.search_category('CE7', lat, lon, 300)
        # 병원 : HP8
        hp8 = kakao.search_category('HP8', lat, lon, 300)
        # 약국 : PM9
        pm9 = kakao.search_category('PM9', lat, lon, 300)

        category = {"CS2": cs2["documents"], "SC4": sc4["documents"], "SW8": sw8["documents"], "FD6": fd6["documents"], "CE7": ce7["documents"], "HP8": hp8["documents"], "PM9": pm9["documents"]}

        around_data_json['officetels'].append({
            "address": addr,
            "latitude": lat,
            "longitude": lon,
            "category": [category]
        })
    except:     
        print(i, addr)

In [11]:
# around_data_json['officetels'][0]['category'][0]["CS2"]

In [12]:
# json_pile_path = '../data/json/around_data.json'
# with open(json_pile_path, 'w') as f:
#     json.dump(around_data_json, f)

In [5]:
with open('../data/json/around_data.json','r') as f:
    around_data_json = json.load(f)
print(json.dumps(around_data_json, indent="\t"))

NameError: name 'json' is not defined

In [40]:
import pandas as pd 

df = pd.read_json('../data/json/around_data.json', orient = "columns")

In [32]:
# json 형식
# df.iloc['i번째 검색장소']['officetels']['category'][0]['카테고리 코드'][0]
df.iloc[0]['officetels']['category'][0]['CS2'][0]

[{'address_name': '서울 강남구 삼성동 143-28',
  'category_group_code': 'CS2',
  'category_group_name': '편의점',
  'category_name': '가정,생활 > 편의점 > GS25',
  'distance': '',
  'id': '8000742',
  'phone': '02-557-2535',
  'place_name': 'GS25 강남동원점',
  'place_url': 'http://place.map.kakao.com/8000742',
  'road_address_name': '서울 강남구 테헤란로77길 7',
  'x': '127.054563985628',
  'y': '37.5069132164195'},
 {'address_name': '서울 노원구 상계동 666-3',
  'category_group_code': 'CS2',
  'category_group_name': '편의점',
  'category_name': '가정,생활 > 편의점',
  'distance': '',
  'id': '1582386948',
  'phone': '02-951-6465',
  'place_name': '스파편의점',
  'place_url': 'http://place.map.kakao.com/1582386948',
  'road_address_name': '서울 노원구 동일로 1493',
  'x': '127.057997596399',
  'y': '37.6614661394716'},
 {'address_name': '서울 종로구 화동 61',
  'category_group_code': 'CS2',
  'category_group_name': '편의점',
  'category_name': '가정,생활 > 편의점 > 이마트24',
  'distance': '',
  'id': '1249602349',
  'phone': '070-7700-2750',
  'place_name': '이마트24 삼

In [41]:
df

,officetels
0,"{'address': '강릉대로 285', 'latitude': 37.7627747..."
1,"{'address': '사임당로 180', 'latitude': 37.7609537..."
2,"{'address': '경강로2326번길 10', 'latitude': 37.767..."
3,"{'address': '선수촌로63번길 18-15', 'latitude': 37.7..."
4,"{'address': '중앙로 190', 'latitude': 38.20735539..."
...,...
9745,"{'address': '남문7길 18', 'latitude': 36.71462937..."
9746,"{'address': '온천대로 832-4', 'latitude': 36.77663..."
9747,"{'address': '시민로440번길 10', 'latitude': 36.7876..."
9748,"{'address': '화랑4안길 19', 'latitude': 36.9926372..."
